![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.6.Clinical_Deidentification_in_Italian.ipynb)

# Clinical Deidentification in Italian

**Protected Health Information**:

Individual’s past, present, or future physical or mental health or condition
provision of health care to the individual
past, present, or future payment for the health care
Protected health information includes many common identifiers (e.g., name, address, birth date, Social Security Number) when they can be associated with the health information.

In [ ]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [3]:
import sys
import os
import json
import pandas as pd
import string
import numpy as np

import sparknlp
import sparknlp_jsl

from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import ResourceDownloader
from sparknlp.util import *
from sparknlp_jsl.annotator import *

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(SECRET, params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 3.4.2
Spark NLP_JSL Version : 3.5.0


# 1. Italian NER Deidentification Models
We have two different models you can use:
* `ner_deid_generic`, detects 8 entities
* `ner_deid_subentity`, detects 19 entities

### Creating pipeline

In [4]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencerDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx") \
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("w2v_cc_300d", "it")\
    .setInputCols(["document","token"])\
	  .setOutputCol("embeddings")

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


## 1.1. NER Deid Generic

**`ner_deid_generic`** extracts:
- Name
- Profession
- Age
- Date
- Contact (Telephone numbers, Email addresses)
- Location (Address, City, Postal code, Hospital Name, Organization)
- ID (Social Security numbers, Medical record numbers)
- Sex

In [5]:
ner_generic = MedicalNerModel.pretrained("ner_deid_generic", "it", "clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_deid_generic")

ner_converter_generic = NerConverter()\
    .setInputCols(["sentence","token","ner_deid_generic"])\
    .setOutputCol("ner_chunk_generic")

ner_deid_generic download started this may take some time.
Approximate size to download 14.3 MB
[OK!]


In [6]:
ner_generic.getClasses()

['O',
 'I-LOCATION',
 'I-CONTACT',
 'I-PROFESSION',
 'I-NAME',
 'I-DATE',
 'B-ID',
 'B-CONTACT',
 'B-PROFESSION',
 'I-ID',
 'B-NAME',
 'B-DATE',
 'B-LOCATION',
 'B-SEX',
 'I-SEX',
 'B-AGE']

## 1.2. NER Deid Subentity

**`ner_deid_subentity`** extracts:

- Patient
- Doctor
- Hospital
- Date
- Organization
- City
- Street
- Username
- Profession
- Phone
- Country
- Age
- Sex
- Email
- ZIP
- Medical Record Number
- Social Security Number
- ID Number
- URL

In [7]:
ner_subentity = MedicalNerModel.pretrained("ner_deid_subentity", "it", "clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_deid_subentity")

ner_converter_subentity = NerConverter()\
    .setInputCols(["sentence", "token", "ner_deid_subentity"])\
    .setOutputCol("ner_chunk_subentity")

ner_deid_subentity download started this may take some time.
Approximate size to download 14.3 MB
[OK!]


In [ ]:
ner_subentity.getClasses()

['O',
 'B-MEDICALRECORD',
 'B-ORGANIZATION',
 'I-PROFESSION',
 'B-DOCTOR',
 'B-USERNAME',
 'B-PROFESSION',
 'B-URL',
 'I-URL',
 'B-CITY',
 'B-DATE',
 'I-MEDICALRECORD',
 'B-SEX',
 'B-PATIENT',
 'I-SEX',
 'I-DOCTOR',
 'I-CITY',
 'B-SSN',
 'I-DATE',
 'I-SSN',
 'B-COUNTRY',
 'B-ZIP',
 'I-STREET',
 'I-PATIENT',
 'B-PHONE',
 'I-PHONE',
 'B-HOSPITAL',
 'B-EMAIL',
 'B-IDNUM',
 'B-STREET',
 'I-IDNUM',
 'I-ORGANIZATION',
 'I-HOSPITAL',
 'B-AGE',
 'I-COUNTRY']

## 1.3. Pipeline

In [8]:
nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentencerDL,
      tokenizer,
      word_embeddings,
      ner_generic,
      ner_converter_generic,
      ner_subentity,
      ner_converter_subentity,
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [9]:
text = "Ho visto Gastone Montanariello (49 anni), virologo, riferito all' Ospedale San Camillo per diabete mal controllato con sintomi risalenti a marzo 2015."

text_df = spark.createDataFrame([[text]]).toDF("text")

result = model.transform(text_df)

### Results for `ner_generic`

In [10]:
result.select(F.explode(F.arrays_zip('ner_chunk_generic.result', 'ner_chunk_generic.metadata')).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+---------------------+----------+
|chunk                |ner_label |
+---------------------+----------+
|Gastone Montanariello|NAME      |
|49                   |AGE       |
|virologo             |PROFESSION|
|Ospedale San Camillo |LOCATION  |
|marzo 2015           |DATE      |
+---------------------+----------+



### Results for `ner_subentity`

In [11]:
result.select(F.explode(F.arrays_zip('ner_chunk_subentity.result', 'ner_chunk_subentity.metadata')).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+---------------------+----------+
|chunk                |ner_label |
+---------------------+----------+
|Gastone Montanariello|PATIENT   |
|49                   |AGE       |
|virologo             |PROFESSION|
|Ospedale San Camillo |HOSPITAL  |
|marzo 2015           |DATE      |
+---------------------+----------+



## DeIdentification

### Obfuscation mode

In [12]:
# Downloading faker entity list.
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/obfuscate_it.txt

In [13]:
deid_masked_entity = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_entity")\
    .setMode("mask")\
    .setMaskingPolicy("entity_labels")

deid_masked_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_chars")\
    .setMode("mask")\
    .setMaskingPolicy("same_length_chars")

deid_masked_fixed_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_fixed_length_chars")\
    .setMode("mask")\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)

deid_obfuscated = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate")\
    .setObfuscateDate(True)\
    .setObfuscateRefFile('obfuscate_it.txt')\
    .setObfuscateRefSource("file")

In [14]:
nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentencerDL,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter_subentity,
      deid_masked_entity,
      deid_masked_char,
      deid_masked_fixed_char,
      deid_obfuscated
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [15]:
deid_lp = LightPipeline(model)

In [16]:
text = "Ho visto Gastone Montanariello (49 anni), virologo, riferito all' Ospedale San Camillo per diabete mal controllato con sintomi risalenti a marzo 2015."

In [17]:
result = deid_lp.annotate(text)

print("\n".join(result['masked_with_entity']))
print("\n")
print("\n".join(result['masked_with_chars']))
print("\n")
print("\n".join(result['masked_fixed_length_chars']))
print("\n")
print("\n".join(result['obfuscated']))

Ho visto <PATIENT> (<AGE> anni), <PROFESSION>, riferito all' <HOSPITAL> per diabete mal controllato con sintomi risalenti a <DATE>.


Ho visto [*******************] (** anni), [******], riferito all' [******************] per diabete mal controllato con sintomi risalenti a [********].


Ho visto **** (**** anni), ****, riferito all' **** per diabete mal controllato con sintomi risalenti a ****.


Ho visto Calogero (11 anni), Micologine., riferito all' AZIENDA SANITARIA PROVINCIALE DI SIRACUSA per diabete mal controllato con sintomi risalenti a 01-04-2000.


In [18]:
pd.set_option("display.max_colwidth", 200)

df = pd.DataFrame(list(zip(result["masked_with_entity"], 
                           result["masked_with_chars"],
                           result["masked_fixed_length_chars"], 
                           result["obfuscated"])),
                 columns= ["Masked_with_entity", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Masked_with_entity,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,"Ho visto <PATIENT> (<AGE> anni), <PROFESSION>, riferito all' <HOSPITAL> per diabete mal controllato con sintomi risalenti a <DATE>.","Ho visto [*******************] (** anni), [******], riferito all' [******************] per diabete mal controllato con sintomi risalenti a [********].","Ho visto **** (**** anni), ****, riferito all' **** per diabete mal controllato con sintomi risalenti a ****.","Ho visto Calogero (11 anni), Micologine., riferito all' AZIENDA SANITARIA PROVINCIALE DI SIRACUSA per diabete mal controllato con sintomi risalenti a 01-04-2000."


### Faker mode

In [19]:
deid_masked_entity = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_entity")\
    .setMode("mask")\
    .setMaskingPolicy("entity_labels")

deid_masked_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_chars")\
    .setMode("mask")\
    .setMaskingPolicy("same_length_chars")

deid_masked_fixed_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_fixed_length_chars")\
    .setMode("mask")\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)

deid_obfuscated = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate")\
    .setLanguage('it')\
    .setObfuscateDate(True)\
    .setObfuscateRefSource('faker')

In [20]:
nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentencerDL,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter_subentity,
      deid_masked_entity,
      deid_masked_char,
      deid_masked_fixed_char,
      deid_obfuscated
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [21]:
deid_lp = LightPipeline(model)

In [22]:
text = "Ho visto Gastone Montanariello (49 anni), virologo, riferito all' Ospedale San Camillo per diabete mal controllato con sintomi risalenti a marzo 2015."

In [23]:
result = deid_lp.annotate(text)

print("\n".join(result['masked_with_entity']))
print("\n")
print("\n".join(result['masked_with_chars']))
print("\n")
print("\n".join(result['masked_fixed_length_chars']))
print("\n")
print("\n".join(result['obfuscated']))

Ho visto <PATIENT> (<AGE> anni), <PROFESSION>, riferito all' <HOSPITAL> per diabete mal controllato con sintomi risalenti a <DATE>.


Ho visto [*******************] (** anni), [******], riferito all' [******************] per diabete mal controllato con sintomi risalenti a [********].


Ho visto **** (**** anni), ****, riferito all' **** per diabete mal controllato con sintomi risalenti a ****.


Ho visto Edgar Kalata (3 anni), Designer, industrial/product, riferito all' PARMER MEDICAL CENTER per diabete mal controllato con sintomi risalenti a 10-20-1981.


In [24]:
pd.set_option("display.max_colwidth", 200)

df = pd.DataFrame(list(zip(result["masked_with_entity"], 
                           result["masked_with_chars"],
                           result["masked_fixed_length_chars"], 
                           result["obfuscated"])),
                 columns= ["Masked_with_entity", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Masked_with_entity,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,"Ho visto <PATIENT> (<AGE> anni), <PROFESSION>, riferito all' <HOSPITAL> per diabete mal controllato con sintomi risalenti a <DATE>.","Ho visto [*******************] (** anni), [******], riferito all' [******************] per diabete mal controllato con sintomi risalenti a [********].","Ho visto **** (**** anni), ****, riferito all' **** per diabete mal controllato con sintomi risalenti a ****.","Ho visto Edgar Kalata (3 anni), Designer, industrial/product, riferito all' PARMER MEDICAL CENTER per diabete mal controllato con sintomi risalenti a 10-20-1981."


# 2. Pretrained Italian Deidentification Pipeline

- We developed a clinical deidentification pretrained pipeline that can be used to deidentify PHI information from Italian medical texts. The PHI information will be masked and obfuscated in the resulting text. 
- The pipeline can mask and obfuscate:
    - Patient
    - Doctor
    - Hospital
    - Date
    - Organization
    - Sex
    - City
    - Street
    - Country
    - ZIP
    - Username
    - Profession
    - Phone
    - Email
    - Age
    - ID number
    - Medical record number
    - Account number
    - SSN
    - Plate Number
    - IP address
    - URL

In [25]:
from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = PretrainedPipeline("clinical_deidentification", "it", "clinical/models")

clinical_deidentification download started this may take some time.
Approx size to download 1.2 GB
[OK!]


In [26]:
text = """RAPPORTO DI RICOVERO
NOME: Lodovico Fibonacci
CODICE FISCALE: MVANSK92F09W408A
INDIRIZZO: Viale Burcardo 7
CITTÀ : Napoli
CODICE POSTALE: 80139
DATA DI NASCITA: 03/03/1946
ETÀ: 70 anni 
SESSO: M
EMAIL: lpizzo@tim.it
DATA DI AMMISSIONE: 12/12/2016
DOTTORE: Eva Viviani
RAPPORTO CLINICO: 70 anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno.
È stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente in seguito, con un'evacuazione normale.
L'esame fisico ha mostrato buone condizioni generali, con addome e genitali normali; l'esame digitale rettale era coerente con un adenoma prostatico di grado I/IV.
L'analisi delle urine ha mostrato 4 globuli rossi/campo e 0-5 leucociti/campo; il resto del sedimento era normale.
L'emocromo è normale; la biochimica ha mostrato una glicemia di 169 mg/dl e trigliceridi 456 mg/dl; la funzione epatica e renale sono normali. PSA di 1,16 ng/ml.

INDIRIZZATO A: Dott. Bruno Ferrabosco - ASL Napoli 1 Centro, Dipartimento di Endocrinologia e Nutrizione - Stretto Scamarcio 320, 80138 Napoli
EMAIL: bferrabosco@poste.it
"""

result = deid_pipeline.annotate(text)
print("\n".join(result['masked_with_chars']))
print("\n")
print("\n".join(result['masked']))
print("\n")
print("\n".join(result['masked_fixed_length_chars']))
print("\n")
print("\n".join(result['obfuscated']))

RAPPORTO DI RICOVERO
NOME: [****************]
CODICE FISCALE: [**************]
INDIRIZZO: [**************]
CITTÀ : [****]
CODICE POSTALE: [***]DATA DI NASCITA: [********]
ETÀ: **anni 
SESSO: *
EMAIL: [***********]
DATA DI AMMISSIONE: [********]
DOTTORE: [*********]
RAPPORTO CLINICO: **anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno.
È stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente in seguito, con un'evacuazione normale.
L'esame fisico ha mostrato buone condizioni generali, con addome e genitali normali; l'esame digitale rettale era coerente con un adenoma prostatico di grado I/IV.
L'analisi delle urine ha mostrato 4 gl

The results can also be inspected vertically by creating a Pandas dataframe as such:

In [ ]:
pd.set_option("display.max_colwidth", None)

df = pd.DataFrame(list(zip(result["sentence"], 
                           result["masked"],
                           result["masked_with_chars"], 
                           result["masked_fixed_length_chars"], 
                           result["obfuscated"])),
                 columns= ["Sentence", "Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Sentence,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,RAPPORTO DI RICOVERO,RAPPORTO DI RICOVERO,RAPPORTO DI RICOVERO,RAPPORTO DI RICOVERO,RAPPORTO DI RICOVERO
1,NOME: Lodovico Fibonacci,NOME: <PATIENT>,NOME: [****************],NOME: ****,NOME: Scotto-Polani
2,CODICE FISCALE: MVANSK92F09W408A,CODICE FISCALE: <SSN>,CODICE FISCALE: [**************],CODICE FISCALE: ****,CODICE FISCALE: ECI-QLN77G15L455Y
3,INDIRIZZO: Viale Burcardo 7\nCITTÀ : Napoli,INDIRIZZO: <STREET>\nCITTÀ : <CITY>,INDIRIZZO: [**************]\nCITTÀ : [****],INDIRIZZO: ****\nCITTÀ : ****,INDIRIZZO: Viale Orlando 808\nCITTÀ : Sesto Raimondo
4,CODICE POSTALE: 80139\nDATA DI NASCITA: 03/03/1946\nETÀ: 70 anni,CODICE POSTALE: <ZIP>DATA DI NASCITA: <DATE>\nETÀ: <AGE>anni,CODICE POSTALE: [***]DATA DI NASCITA: [********]\nETÀ: **anni,CODICE POSTALE: ****DATA DI NASCITA: ****\nETÀ: ****anni,CODICE POSTALE: 53581DATA DI NASCITA: 19/04/1946\nETÀ: 5anni
5,SESSO: M\nEMAIL: lpizzo@tim.it\nDATA DI AMMISSIONE: 12/12/2016,SESSO: <SEX>\nEMAIL: <E-MAIL>\nDATA DI AMMISSIONE: <DATE>,SESSO: *\nEMAIL: [***********]\nDATA DI AMMISSIONE: [********],SESSO: ****\nEMAIL: ****\nDATA DI AMMISSIONE: ****,SESSO: U\nEMAIL: HenryWatson@world.com\nDATA DI AMMISSIONE: 30/01/2017
6,DOTTORE: Eva Viviani,DOTTORE: <DOCTOR>,DOTTORE: [*********],DOTTORE: ****,DOTTORE: Sig. Fredo Marangoni
7,"RAPPORTO CLINICO: 70 anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno.","RAPPORTO CLINICO: <AGE>anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno.","RAPPORTO CLINICO: **anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno.","RAPPORTO CLINICO: ****anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno.","RAPPORTO CLINICO: 5anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno."
8,"È stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente in seguito, con un'evacuazione normale.","È stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente in seguito, con un'evacuazione normale.","È stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente in seguito, con un'evacuazione normale.","È stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente in seguito, con un'evacuazione normale.","È stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente 